In [63]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
pd.options.mode.chained_assignment = None  # default='warn'

METADATA = {'Contributor': 'T. Dunn'}
SAVEFIGS = False

df0 = pd.read_csv('./monster-saves.csv')
df0 = df0.astype({'Book': 'category', 'Category': 'category', 'Legendary': 'category', 'Type': 'category', 'Size': 'category'})
df0 = df0.astype({'AC': 'int32', 'adj AC': 'int32'})

df0 = pd.read_csv('./monster-saves.csv')

dfD = pd.read_csv('../../assets/data/dmg-targets.csv') # 'CR','PB','AC','AB','DC'

print('Includes {} monsters in database'.format(len(df0.index.to_list())))

Includes 1436 monsters in database


In [4]:
# Setup template and html config for plotly figures.
import plotly.graph_objects as go

TFB_TEMPLATE = dict(
    layout=go.Layout(
        template='plotly_white',
        autosize=True, # must be True to auto-scale when resizing
        margin=dict(l=50, r=25, b=40, t=20, pad=4),
        font=dict(
            family='sans-serif',
            size=14
        ),
        hovermode='closest',
        xaxis=dict(
            automargin=False,
            showline=True,
            linecolor='#444',
            linewidth=2,
            mirror=True,
            tickmode='linear',
            ticks='outside',
            minor=dict(
                tickmode='linear',
                ticks='outside',
            ),
            zeroline=False,
        ),
        yaxis=dict(
            automargin=False,
            showline=True,
            linecolor='#444',
            linewidth=2,
            mirror=True,
            ticks='outside',
            minor=dict(
                ticks='outside',
            ),
            zeroline=False,
        ),
        hoverlabel=dict(align='left'),
    )
)

TFB_CONFIG = {
    'responsive': True, # must be True to auto-scale when resizing
    'autosizable': True, # doesn't impact auto rescaling
    'showAxisDragHandles': False,
    'displaylogo': False,
    'displayModeBar': 'hover',
    'modeBarButtonsToRemove': [
        'select2d',
        'lasso2d',
        'zoom2d',
        'zoomIn2d',
        'zoomOut2d',
        'pan2d',
        'autoScale2d',
        'hoverClosestCartesian',
        'hoverCompareCartesian',
        'toggleSpikelines',
        'resetScale2d',
    ],
    'toImageButtonOptions': {
        'format': 'png', # one of png, svg, jpeg, webp
        'filename': 'tfb-plot',
        'height': 450,
        'width': 600,
        'scale': 2
    },
}

In [64]:
def average_ability_mod(row):
    return (row['Str Mod'] + row['Dex Mod'] + row['Con Mod'] + row['Int Mod'] + row['Wis Mod'] + row['Cha Mod'])/6.0

def average_save_bonus(row):
    return (row['Str Save'] + row['Dex Save'] + row['Con Save'] + row['Int Save'] + row['Wis Save'] + row['Cha Save'])/6.0

def adjusted_AC(row):
    adjAC = 14 + (row['Str Save'] + row['Dex Save'] + row['Con Save'] + row['Int Save'] + row['Wis Save'] + row['Cha Save'])/6.0
    adjAC = (adjAC + row['AC'])/2.0
    return adjAC

def adjusted_AC2(row):
    adjAC = 14 + (row['Str Mod'] + row['Dex Mod'] + row['Con Mod'] + row['Int Mod'] + row['Wis Mod'] + row['Cha Mod'])/6.0
    adjAC = (adjAC + row['AC'])/2.0
    return adjAC

def nProficiencies(row):
    n = row['Str Prof'] + row['Dex Prof'] + row['Con Prof'] + row['Int Prof'] + row['Wis Prof'] + row['Cha Prof']
    return n

In [70]:
# Saving throw bonus
columns = ['Str Save','Dex Save','Con Save','Int Save','Wis Save','Cha Save','Ave Save','AC - 14']
colors = ['#d62728','#9467bd','#8c564b','#1f77b4','#2ca02c','#ff7f0e','gray','black']

dfD = pd.read_csv('../../assets/data/dmg-targets.csv') # 'CR','PB','AC','AB','DC'
dfD = dfD.set_index('CR')

df1 = df0[df0['Category'].isin(['generic','legendary','unique']) & df0['CR'].between(1, 30)]
df1['Ave Save'] = (df1['Str Save'] + df1['Dex Save'] + df1['Con Save'] + df1['Int Save'] + df1['Wis Save'] + df1['Cha Save'])/6
df1['AC - 14'] = df1.apply(lambda row: dfD.loc[row['CR'], 'AC'] - 14, axis=1)
#df1['AC - 13'] = df1['AC'] - 13
#df1['CR'] = 2 + 3*np.floor((df1['CR'] - 1)/3.0)
dfG = df1[['CR'] + columns].groupby('CR').mean()

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=TFB_TEMPLATE,
        margin=dict(l=60, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='challenge rating',
            range=[-1,31],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='saving throw bonus',
            range=[-2,16],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

# plot data
x = dfG.index.to_series()
for column, c in zip(columns, colors):
    coefs = np.polyfit(x, dfG[column], 1)
    poly = np.poly1d(coefs)
    fig.add_trace(go.Scatter(
        x=dfG.index.to_series(), 
        y=dfG[column],
        mode='markers', 
        marker=dict(size=6, color=c),
        name=column,
        legendgroup=column,
        hovertemplate=
            'CR %{x}<br>'+
            'Bonus %{y:,.1f}' + 
            '<extra></extra>'
    ))
    fig.add_trace(go.Scatter(
        x=x, 
        y=poly(x),
        mode='lines', 
        marker=dict(color=c),
        name=column,
        legendgroup=column,
        showlegend=False,
        hoverinfo='skip',
    ))

# show figure
fig.show(config=TFB_CONFIG)

# save large format figure
if SAVEFIGS:
    file_name = f'./fig-monster-save-bonus-vs-cr-large.html'
    fig_html = fig.to_html(
        config=TFB_CONFIG,
        include_plotlyjs=False, 
        full_html=False, 
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = 'plotly-div-large'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))

# save small format figure
if SAVEFIGS:
    fig.update_traces(marker=dict(size=4))
    fig.update_layout(font=dict(size=10))
    file_name = f'./fig-monster-save-bonus-vs-cr-small.html'
    fig_html = fig.to_html(
        config=TFB_CONFIG,
        include_plotlyjs=False, 
        full_html=False, 
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = 'plotly-div-small'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))

In [67]:
# Saving throw modifier
columns = ['Str Mod','Dex Mod','Con Mod','Int Mod','Wis Mod','Cha Mod','Ave Mod','AC - 14']
colors = ['#d62728','#9467bd','#8c564b','#1f77b4','#2ca02c','#ff7f0e','gray','black']

dfD = pd.read_csv('../../assets/data/dmg-targets.csv') # 'CR','PB','AC','AB','DC'
dfD = dfD.set_index('CR')

df1 = df0[df0['Category'].isin(['generic','legendary','unique']) & df0['CR'].between(1, 30)]
df1['Ave Mod'] = (df1['Str Mod'] + df1['Dex Mod'] + df1['Con Mod'] + df1['Int Mod'] + df1['Wis Mod'] + df1['Cha Mod'])/6
df1['AC - 14'] = df1.apply(lambda row: dfD.loc[row['CR'], 'AC'] - 14, axis=1)
#df1['AC - 13'] = df1['AC'] - 13
#df1['CR'] = 2 + 3*np.floor((df1['CR'] - 1)/3.0)
dfG = df1[['CR'] + columns].groupby('CR').mean()

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=TFB_TEMPLATE,
        margin=dict(l=60, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='challenge rating',
            range=[-1,31],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='saving throw modifier',
            range=[-2,16],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

# plot data
x = dfG.index.to_series()
for column, c in zip(columns, colors):
    coefs = np.polyfit(x, dfG[column], 1)
    poly = np.poly1d(coefs)
    fig.add_trace(go.Scatter(
        x=dfG.index.to_series(), 
        y=dfG[column],
        mode='markers', 
        marker=dict(size=6, color=c),
        name=column,
        legendgroup=column,
        hovertemplate=
            'CR %{x}<br>'+
            'Modifier %{y:,.1f}' + 
            '<extra></extra>'
    ))
    fig.add_trace(go.Scatter(
        x=x, 
        y=poly(x),
        mode='lines', 
        marker=dict(color=c),
        name=column,
        legendgroup=column,
        showlegend=False,
        hoverinfo='skip',
    ))

# show figure
fig.show(config=TFB_CONFIG)

# save large format figure
if SAVEFIGS:
    file_name = f'./fig-monster-save-modifier-vs-cr-large.html'
    fig_html = fig.to_html(
        config=TFB_CONFIG,
        include_plotlyjs=False, 
        full_html=False, 
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = 'plotly-div-large'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))

# save small format figure
if SAVEFIGS:
    fig.update_traces(marker=dict(size=4))
    fig.update_layout(font=dict(size=10))
    file_name = f'./fig-monster-save-modifier-vs-cr-small.html'
    fig_html = fig.to_html(
        config=TFB_CONFIG,
        include_plotlyjs=False, 
        full_html=False, 
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = 'plotly-div-small'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))

In [57]:
x = 0.05
n = np.array(range(-6, 7))

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=TFB_TEMPLATE,
        margin=dict(l=60, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='$8 + AB - AC$',
            range=[-6.5,6.5],
            tick0=0, dtick=1,
            #minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='attack hit probability',
            range=[0,1.2],
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.1),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)


#ax.plot(n, np.multiply(0.05, np.add(12, n)), '-', label='1 + n*x', color='k')
#ax.plot(n, np.multiply(0.60, np.power(1.083, n)), '-', label='(1 + x)^n', color='r')

fig.add_trace(go.Scatter(
    x=n, 
    y=np.multiply(0.05, np.add(13, n)),
    mode='markers+lines', 
    name='$1 + n \cdot x$',
    hovertemplate=
        'n %{x}<br>'+
        'Hit Probability %{y:,.0%}' + 
        '<extra></extra>'
))

fig.add_trace(go.Scatter(
    x=n, 
    y=np.multiply(0.65, np.power(1.077, n)),
    mode='markers+lines', 
    name='$(1 + x)^n$',
    hovertemplate=
        'n %{x}<br>'+
        'Hit Probability %{y:,.0%}' + 
        '<extra></extra>'
))

In [51]:
# plots the average probability of hitting with an attack
pc = {
    'level':      [ 1, 2, 3, 4, 5, 6, 7, 8, 9,10,11,12,13,14,15,16,17,18,19,20],
    'attack mod': [ 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
    'prof bonus': [ 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 6, 6, 6, 6],
    "armor class": {
        "Barbarian": [15, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 14, 14, 14, 15, 17],
        "Bard":      [14, 14, 14, 14, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15],
        "Cleric":    [17, 17, 17, 17, 17, 17, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18],
        "Druid":     [15, 15, 15, 16, 16, 16, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18],
        "Fighter":   [16, 16, 16, 16, 16, 16, 16, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18],
        "Monk":      [16, 16, 16, 17, 17, 17, 17, 18, 18, 18, 18, 19, 19, 19, 19, 20, 20, 20, 20, 20],
        "Paladin":   [16, 16, 16, 16, 16, 16, 16, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18],
        "Ranger":    [14, 14, 14, 15, 16, 16, 16, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17],
        "Rogue":     [14, 14, 14, 15, 16, 16, 16, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17],
        "Sorcerer":  [12, 12, 12, 12, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15],
        "Warlock":   [13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13],
        "Wizard":    [12, 12, 12, 12, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15],
    }
}
pc['attack bonus'] = [pc['attack mod'][i] + pc['prof bonus'][i] for i in range(20)]
ac_mean = np.zeros(20)
for c in pc['armor class']:
    ac_mean += np.array(pc['armor class'][c])
ac_mean /= len(pc['armor class'])
pc['armor class'] = list(ac_mean)

dfD = pd.read_csv('../../assets/data/dmg-targets.csv') # 'CR','PB','AC','AB','DC'
dfD = dfD[dfD['CR'].between(1, 20)]

def attack_hit_crit_prob(AC, AB):
    return max(0.05, min(0.95, 0.05*(21 + AB - AC)))

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=TFB_TEMPLATE,
        margin=dict(l=60, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='level',
            range=[0,21],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='attack hit probability',
            range=[0,1],
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.1),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)


# monster vs pc
lvl = np.array(pc['level'])
ac = np.array(pc['armor class'])
ab = np.array(dfD['AB'])
phit = np.array([attack_hit_crit_prob(ac[i], ab[i]) for i in range(20)])
fig.add_trace(go.Scatter(
    x=lvl, 
    y=phit,
    mode='markers+lines', 
    name='monster vs PC',
    hovertemplate=
        'Level %{x}<br>'+
        'Hit Probability %{y:,.0%}' + 
        '<extra></extra>'
))
print(np.mean(phit))

# pc vs monster
lvl = np.array(pc['level'])
ab = np.array(pc['attack bonus'])
ac = np.array(dfD['AC'])
phit = np.array([attack_hit_crit_prob(ac[i], ab[i]) for i in range(20)])
fig.add_trace(go.Scatter(
    x=lvl, 
    y=phit,
    mode='markers+lines', 
    name='PC vs monster',
    hovertemplate=
        'Level %{x}<br>'+
        'Hit Probability %{y:,.0%}' + 
        '<extra></extra>'
))
print(np.mean(phit))

# reference line
fig.add_trace(go.Scatter(
    x=lvl, 
    y=[0.65]*len(lvl),
    mode='lines', 
    name='reference',
    line=dict(color='black', dash='dash'),
    showlegend=False,
    hoverinfo='skip',
))

# show figure
fig.show(config=TFB_CONFIG)

# save large format figure
if SAVEFIGS:
    file_name = f'./fig-attack-hit-probability-vs-level-large.html'
    fig_html = fig.to_html(
        config=TFB_CONFIG,
        include_plotlyjs=False, 
        full_html=False, 
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = 'plotly-div-large'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))

# save small format figure
if SAVEFIGS:
    fig.update_traces(marker=dict(size=4))
    fig.update_layout(font=dict(size=10))
    file_name = f'./fig-attack-hit-probability-vs-level-small.html'
    fig_html = fig.to_html(
        config=TFB_CONFIG,
        include_plotlyjs=False, 
        full_html=False, 
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = 'plotly-div-small'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))

0.6158333333333333
0.6525000000000001


In [58]:
# plots the average probability of hitting with a saving throw

pc = {
    'level':      [ 1, 2, 3, 4, 5, 6, 7, 8, 9,10,11,12,13,14,15,16,17,18,19,20],
    'attack mod': [ 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
    'prof bonus': [ 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 6, 6, 6, 6],
    'ability mod': {
        'ability 1': [ 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
        'ability 2': [ 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
        'ability 3': [ 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
        'ability 4': [-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1],
        'ability 5': [-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1],
        'ability 6': [-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1],
    },
    'save mod': [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
}
pc['save dc'] = [8 + pc['attack mod'][i] + pc['prof bonus'][i] for i in range(20)]
for i in range(20):
    pc['save mod'][i] = np.mean([pc['ability mod'][ability][i] for ability in pc['ability mod']])
pc['save bonus'] = [pc['save mod'][i] + pc['prof bonus'][i]/3 for i in range(20)]

dfD = pd.read_csv('../../assets/data/dmg-targets.csv') # 'CR','PB','AC','AB','DC'
dfD = dfD[dfD['CR'].between(1, 20)]

dfM = pd.read_csv('monster-saves.csv')
dfM = dfM[dfM['CR'].between(1,20)]
dfM['Ave Mod'] = dfM.apply(lambda row: average_ability_mod(row), axis=1)
dfM = dfM[['CR','Ave Mod']].groupby('CR').mean().reset_index()
dfM.sort_values(by='CR', inplace=True)

def save_hit_prob(sb, dc):
    return max(0.05, min(1.00, 0.05*(dc - sb - 1)))

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=TFB_TEMPLATE,
        margin=dict(l=60, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='level',
            range=[0,21],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='save hit probability',
            range=[0,1],
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.1),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)


# monster vs pc
lvl = np.array(pc['level'])
sb = np.array(pc['save bonus'])
dc = np.array(dfD['DC'])
phit = np.array([save_hit_prob(sb[i], dc[i]) for i in range(20)])
fig.add_trace(go.Scatter(
    x=lvl, 
    y=phit,
    mode='markers+lines', 
    name='monster vs PC',
    hovertemplate=
        'Level %{x}<br>'+
        'Hit Probability %{y:,.0%}' + 
        '<extra></extra>'
))
print(np.mean(phit))

# pc vs monster
lvl = np.array(pc['level'])
dc = np.array(pc['save dc'])
sb = np.array(dfM['Ave Mod'])
phit = np.array([save_hit_prob(sb[i], dc[i]) for i in range(20)])
fig.add_trace(go.Scatter(
    x=lvl, 
    y=phit,
    mode='markers+lines', 
    name='PC vs monster',
    hovertemplate=
        'Level %{x}<br>'+
        'Hit Probability %{y:,.0%}' + 
        '<extra></extra>'
))
print(np.mean(phit))

# reference line
fig.add_trace(go.Scatter(
    x=lvl, 
    y=[0.65]*len(lvl),
    mode='lines', 
    name='reference',
    line=dict(color='black', dash='dash'),
    showlegend=False,
    hoverinfo='skip',
))

# show figure
fig.show(config=TFB_CONFIG)

# save large format figure
if SAVEFIGS:
    file_name = f'./fig-save-hit-probability-vs-level-large.html'
    fig_html = fig.to_html(
        config=TFB_CONFIG,
        include_plotlyjs=False, 
        full_html=False, 
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = 'plotly-div-large'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))

# save small format figure
if SAVEFIGS:
    fig.update_traces(marker=dict(size=4))
    fig.update_layout(font=dict(size=10))
    file_name = f'./fig-save-hit-probability-vs-level-small.html'
    fig_html = fig.to_html(
        config=TFB_CONFIG,
        include_plotlyjs=False, 
        full_html=False, 
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = 'plotly-div-small'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))

0.6533333333333333
0.6591635905115173


In [24]:
# plots average save DC 

pc = {
    'level':      [ 1, 2, 3, 4, 5, 6, 7, 8, 9,10,11,12,13,14,15,16,17,18,19,20],
    'attack mod': [ 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
    'prof bonus': [ 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 6, 6, 6, 6],
}
pc['attack bonus'] = [pc['attack mod'][i] + pc['prof bonus'][i] for i in range(20)]
pc['save dc'] = [pc['attack bonus'][i] + 8 for i in range(20)]

def save_fail_prob(SB, DC):
    return max(0.0, min(1.0, 0.05*(DC - SB - 1)))

dfD = pd.read_csv('../../assets/data/dmg-targets.csv') # 'CR','PB','AC','AB','DC'
dfD = dfD[dfD['CR'].between(1, 20)]

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=TFB_TEMPLATE,
        margin=dict(l=60, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='level',
            range=[0,21],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='save DC',
            range=[12,20],
            tick0=0, dtick=1,
            minor=dict(tick0=0, dtick=0.5),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

# plot monster save DCs
fig.add_trace(go.Scatter(
    x=np.array(pc['level']), 
    y=np.array(dfD['DC']),
    mode='markers+lines', 
    name='monsters',
    hovertemplate=
        'Level %{x}<br>'+
        'Save DC %{y:,.0f}' + 
        '<extra></extra>'
))

# plot player character save DCs
fig.add_trace(go.Scatter(
    x=np.array(pc['level']), 
    y=np.array(pc['save dc']),
    mode='markers+lines', 
    name='player characters',
    hovertemplate=
        'Level %{x}<br>'+
        'Save DC %{y:,.0f}' + 
        '<extra></extra>'
))


# show figure
fig.show(config=TFB_CONFIG)

# save large format figure
if SAVEFIGS:
    file_name = f'./fig-save-dc-vs-level-large.html'
    fig_html = fig.to_html(
        config=TFB_CONFIG,
        include_plotlyjs=False, 
        full_html=False, 
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = 'plotly-div-large'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))

# save small format figure
if SAVEFIGS:
    fig.update_traces(marker=dict(size=4))
    fig.update_layout(font=dict(size=10))
    file_name = f'./fig-save-dc-vs-level-small.html'
    fig_html = fig.to_html(
        config=TFB_CONFIG,
        include_plotlyjs=False, 
        full_html=False, 
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = 'plotly-div-small'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))

In [23]:
# plots average save bonus 

pc = {
    'level':      [ 1, 2, 3, 4, 5, 6, 7, 8, 9,10,11,12,13,14,15,16,17,18,19,20],
    'attack mod': [ 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
    'prof bonus': [ 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 6, 6, 6, 6],
    'ability mod': {
        'ability 1': [ 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
        'ability 2': [ 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
        'ability 3': [ 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
        'ability 4': [-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1],
        'ability 5': [-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1],
        'ability 6': [-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1],
    },
    'save mod': [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
}
pc['save dc'] = [8 + pc['attack mod'][i] + pc['prof bonus'][i] for i in range(20)]
for i in range(20):
    pc['save mod'][i] = np.mean([pc['ability mod'][ability][i] for ability in pc['ability mod']])
pc['save bonus'] = [pc['save mod'][i] + pc['prof bonus'][i]/3 for i in range(20)]

dfD = pd.read_csv('../../assets/data/dmg-targets.csv') # 'CR','PB','AC','AB','DC'
dfD = dfD[dfD['CR'].between(1, 20)]

dfM = pd.read_csv('monster-saves.csv')
dfM = dfM[dfM['CR'].between(1,20)]
dfM['Ave Mod'] = dfM.apply(lambda row: average_ability_mod(row), axis=1)
dfM = dfM[['CR','Ave Mod']].groupby('CR').mean().reset_index()
dfM.sort_values(by='CR', inplace=True)

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=TFB_TEMPLATE,
        margin=dict(l=60, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='level',
            range=[0,21],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='save bonus',
            range=[0,5],
            tick0=0, dtick=1,
            minor=dict(tick0=0, dtick=0.5),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)


# plot monster save DCs
fig.add_trace(go.Scatter(
    x=np.array(pc['level']), 
    y=np.array(dfM['Ave Mod']),
    mode='markers+lines', 
    name='monsters',
    hovertemplate=
        'Level %{x}<br>'+
        'Save Bonus %{y:,.0f}' + 
        '<extra></extra>'
))

# plot player character save DCs
fig.add_trace(go.Scatter(
    x=np.array(pc['level']), 
    y=np.array(pc['save bonus']),
    mode='markers+lines', 
    name='player characters',
    hovertemplate=
        'Level %{x}<br>'+
        'Save Bonus %{y:,.0f}' + 
        '<extra></extra>'
))

# show figure
fig.show(config=TFB_CONFIG)

# save large format figure
if SAVEFIGS:
    file_name = f'./fig-save-bonus-vs-level-large.html'
    fig_html = fig.to_html(
        config=TFB_CONFIG,
        include_plotlyjs=False, 
        full_html=False, 
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = 'plotly-div-large'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))

# save small format figure
if SAVEFIGS:
    fig.update_traces(marker=dict(size=4))
    fig.update_layout(font=dict(size=10))
    file_name = f'./fig-save-bonus-vs-level-small.html'
    fig_html = fig.to_html(
        config=TFB_CONFIG,
        include_plotlyjs=False, 
        full_html=False, 
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = 'plotly-div-small'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))

In [22]:
# plots average attack bonus

pc = {
    'level':      [ 1, 2, 3, 4, 5, 6, 7, 8, 9,10,11,12,13,14,15,16,17,18,19,20],
    'attack mod': [ 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
    'prof bonus': [ 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 6, 6, 6, 6],
}
pc['attack bonus'] = [pc['attack mod'][i] + pc['prof bonus'][i] for i in range(20)]
pc['save dc'] = [pc['attack bonus'][i] + 8 for i in range(20)]

dfD = pd.read_csv('../../assets/data/dmg-targets.csv') # 'CR','PB','AC','AB','DC'
dfD = dfD[dfD['CR'].between(1, 20)]

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=TFB_TEMPLATE,
        margin=dict(l=60, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='level',
            range=[0,21],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='attack bonus',
            range=[2,12],
            tick0=0, dtick=1,
            minor=dict(tick0=0, dtick=0.5),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

# plot monster save DCs
fig.add_trace(go.Scatter(
    x=np.array(pc['level']), 
    y=np.array(dfD['AB']),
    mode='markers+lines', 
    name='monsters',
    hovertemplate=
        'Level %{x}<br>'+
        'Attack Bonus %{y:,.0f}' + 
        '<extra></extra>'
))

# plot player character save DCs
fig.add_trace(go.Scatter(
    x=np.array(pc['level']), 
    y=np.array(pc['attack bonus']),
    mode='markers+lines', 
    name='player characters',
    hovertemplate=
        'Level %{x}<br>'+
        'Attack Bonus %{y:,.0f}' + 
        '<extra></extra>'
))

# show figure
fig.show(config=TFB_CONFIG)

# save large format figure
if SAVEFIGS:
    file_name = f'./fig-attack-bonus-vs-level-large.html'
    fig_html = fig.to_html(
        config=TFB_CONFIG,
        include_plotlyjs=False, 
        full_html=False, 
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = 'plotly-div-large'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))

# save small format figure
if SAVEFIGS:
    fig.update_traces(marker=dict(size=4))
    fig.update_layout(font=dict(size=10))
    file_name = f'./fig-attack-bonus-vs-level-small.html'
    fig_html = fig.to_html(
        config=TFB_CONFIG,
        include_plotlyjs=False, 
        full_html=False, 
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = 'plotly-div-small'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))

In [21]:
# plots average armor class 

pc = {
    'level':      [ 1, 2, 3, 4, 5, 6, 7, 8, 9,10,11,12,13,14,15,16,17,18,19,20],
    'attack mod': [ 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
    'prof bonus': [ 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 6, 6, 6, 6],
    "armor class": {
        "Barbarian": [15, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 14, 14, 14, 15, 17],
        "Bard":      [14, 14, 14, 14, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15],
        "Cleric":    [17, 17, 17, 17, 17, 17, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18],
        "Druid":     [15, 15, 15, 16, 16, 16, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18],
        "Fighter":   [16, 16, 16, 16, 16, 16, 16, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18],
        "Monk":      [16, 16, 16, 17, 17, 17, 17, 18, 18, 18, 18, 19, 19, 19, 19, 20, 20, 20, 20, 20],
        "Paladin":   [16, 16, 16, 16, 16, 16, 16, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18],
        "Ranger":    [14, 14, 14, 15, 16, 16, 16, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17],
        "Rogue":     [14, 14, 14, 15, 16, 16, 16, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17],
        "Sorcerer":  [12, 12, 12, 12, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15],
        "Warlock":   [13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13],
        "Wizard":    [12, 12, 12, 12, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15],
    }
}
ac_mean = np.zeros(20)
for c in pc['armor class']:
    ac_mean += np.array(pc['armor class'][c])
ac_mean /= len(pc['armor class'])
pc['armor class'] = list(ac_mean)

dfD = pd.read_csv('../../assets/data/dmg-targets.csv') # 'CR','PB','AC','AB','DC'
dfD = dfD[dfD['CR'].between(1, 20)]

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=TFB_TEMPLATE,
        margin=dict(l=60, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='level',
            range=[0,21],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='armor class',
            range=[12,20],
            tick0=0, dtick=1,
            minor=dict(tick0=0, dtick=0.5),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

# plot monster save DCs
fig.add_trace(go.Scatter(
    x=np.array(pc['level']), 
    y=np.array(dfD['AC']),
    mode='markers+lines', 
    name='monsters',
    hovertemplate=
        'Level %{x}<br>'+
        'Armor Class %{y:,.0f}' + 
        '<extra></extra>'
))

# plot player character save DCs
fig.add_trace(go.Scatter(
    x=np.array(pc['level']), 
    y=np.array(pc['armor class']),
    mode='markers+lines', 
    name='player characters',
    hovertemplate=
        'Level %{x}<br>'+
        'Armor Class %{y:,.0f}' + 
        '<extra></extra>'
))

# show figure
fig.show(config=TFB_CONFIG)

# save large format figure
if SAVEFIGS:
    file_name = f'./fig-armor-class-vs-level-large.html'
    fig_html = fig.to_html(
        config=TFB_CONFIG,
        include_plotlyjs=False, 
        full_html=False, 
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = 'plotly-div-large'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))

# save small format figure
if SAVEFIGS:
    fig.update_traces(marker=dict(size=4))
    fig.update_layout(font=dict(size=10))
    file_name = f'./fig-armor-class-vs-level-small.html'
    fig_html = fig.to_html(
        config=TFB_CONFIG,
        include_plotlyjs=False, 
        full_html=False, 
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = 'plotly-div-small'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))